<a href="https://colab.research.google.com/github/Jaywestty/News-Crime-Classification/blob/main/News_Text_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **NEWS TEXT SUMMARIZER PROJECT**

####**Project Description:**
This project aims to automatically summarize news articles into concise, factual highlights using Hugging Face Transformers. The summarization model is based on the bart-base architecture, chosen for its strong performance on abstractive summarization while remaining lightweight enough to run within Google Colab's free-tier resource limits. The dataset, sourced from Hugging Face’s public datasets repository, contains diverse news articles for training and evaluation. The system is designed to generate short, accurate, and easily readable summaries that retain the key points of the original article, making it useful for quick news consumption.

#### **Install dependecies**

In [ ]:
!pip install transformers datasets evaluate rouge_score accelerate nltk -q

#### **Import required libraries**

In [ ]:
from datasets import load_dataset
from transformers import BartForConditionalGeneration, BartTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import numpy as np
import torch
import nltk
import gc
import evaluate
nltk.download('punkt')

In [ ]:
#Clear up memory to aid colab limit
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

#### **Load BART Tokenizer and Model**  

For fine-tuning, we load the **BART-base** model and tokenizer directly from Hugging Face.  

- **BART-base** is chosen over **T5-small** because:  
  - It generally produces **higher-quality summaries**.  
  - It balances performance with efficiency, making it suitable for **Colab free tier GPUs**.  


In [ ]:
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

print(f"Model loaded! Parameters: {model.num_parameters():,}")
clear_memory()

#### **Load Dataset**

We use the **CNN/DailyMail dataset** provided by Hugging Face Datasets.  

Due to restricted GPU access on Colab, we work with a **subset**:  
- 8,000 samples from the **training set**  
- 800 samples from the **validation set**  
- 800 samples from the **test set**  

In [ ]:
print("Loading CNN-DailyMail dataset...")
dataset = load_dataset('cnn_dailymail', '3.0.0')

print("Sample article:\n", dataset['train'][0]['article'][:200])
print("\nSample summary:\n", dataset['train'][0]['highlights'])

# Reduce dataset for Colab constraints
train_dataset = dataset['train'].select(range(8000))  # Slightly smaller for BART
val_dataset = dataset['validation'].select(range(800))
test_dataset = dataset['test'].select(range(800))

print(f"Dataset sizes - Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

#### **BART-specific preprocessing**

## Data Preprocessing  
Before training, we need to prepare the text for BART:  
- Tokenize the input and target texts  
- Truncate or pad sequences to a fixed length  
- Format inputs and labels for Seq2Seq training  



In [ ]:
max_input_length = 1024  # BART can handle longer inputs
max_target_length = 142  # CNN-DM standard summary length

def preprocess(example):
    model_inputs = tokenizer(
        example['article'],
        max_length=max_input_length,
        truncation=True,
        padding="max_length"  # Changed from True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example['highlights'],
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )

    # Replace pad token id with -100 for label loss masking
    labels_ids = labels["input_ids"]
    labels_ids = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels_ids
    ]
    model_inputs["labels"] = labels_ids
    return model_inputs


print("Preprocessing datasets...")
train_tokenized = train_dataset.map(preprocess, batched=True, remove_columns=train_dataset.column_names)
val_tokenized = val_dataset.map(preprocess, batched=True, remove_columns=val_dataset.column_names)

clear_memory()

In [ ]:
print(train_tokenized)

In [ ]:
print("Pad token ID:", tokenizer.pad_token_id)
print("Vocab size:", tokenizer.vocab_size)


#### **Load ROUGE for Evaluation**


In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = [[(token if token != -100 else tokenizer.pad_token_id) for token in label] for label in labels]
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=predictions, references=labels, use_stemmer=True)
    return {k: round(v * 100, 2) for k, v in result.items()}

#### **Train the Model**

The model is trained on the reduced dataset:  
- Training loss is logged  
- Validation loss is tracked for overfitting  
- Best checkpoint is saved  


In [ ]:
#Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-news-summarizer",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,   # Increase to 3 (sweet spot for Colab free tier)
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir='./logs',
    logging_steps=100,   # Log progress every 100 steps
    save_strategy="epoch"  # Save at the end of each epoch
)


In [ ]:
#Trainer Setup
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
clear_memory()
trainer.train()

#### **Best Model Selection**  

After training, **epoch 2** was identified as the best-performing checkpoint.  

- **Training Loss:** 1.60  
- **Validation Loss:** 2.23  
- **ROUGE-1:** 24.6  
- **ROUGE-2:** 9.5  
- **ROUGE-Lsum:** 22.48  

These results show that by the second epoch, the model had already reached an optimal balance between **low loss** and **high ROUGE scores**, indicating strong summarization quality without overfitting.  

For this reason, **epoch 2 is saved as the final model checkpoint**, which will be used for deployment.  


#### **Evaluation of Model**

We evaluate the model using:  
- **ROUGE scores** (ROUGE-1, ROUGE-2, ROUGE-L)

In [ ]:
metrics = trainer.evaluate()
print(metrics)

#### **Testing Best Checkpoint (Epoch 2)  on an article**

Before saving, we reload the **epoch 2 checkpoint** to verify its performance on a sample article.  
This step ensures that the model generates **coherent and concise summaries** before committing it as our final saved version.  

By testing on real input text, we can confirm that the chosen checkpoint generalizes well beyond the validation set.  


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Point directly to epoch 2 checkpoint
model_path = "./bart-news-summarizer/checkpoint-8000"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
print(model)

In [ ]:
import re

def clean_and_merge_article(article):
    # Step 1: Clean article text
    article = re.sub(r"\s+", " ", article.strip())  # collapse spaces & newlines
    article = article.replace(" ,", ",").replace(" .", ".")  # fix space before punctuation

    # Step 2: Summarize using your model
    inputs = tokenizer(article, return_tensors="pt", max_length=1024, truncation=True).to(model.device)
    summary_ids = model.generate(**inputs, max_length=200, min_length=80, length_penalty=2.0, num_beams=4)
    raw_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Step 3: Merge summary into one sentence
    summary = re.sub(r'\s+', ' ', raw_summary.strip())
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', summary)
    sentences = [s.strip(" .") for s in sentences if s.strip()]

    if not sentences:
        return ""
    if len(sentences) == 1:
        return sentences[0] + "."

    merged = ", ".join(sentences[:-1]) + " and " + sentences[-1]
    return merged.strip() + "."


In [ ]:
article = """
The Vice Chancellor, Federal University of Technology and Environmental Sciences, Iyin Ekiti, Prof. Gbenga Aribisala, has said that the new institution will begin the admission process in September.

Aribisala said that the admission process would follow the National Universities Commission Resource Verification exercise taking place soon in the university.

The VC, who spoke in Ado Ekiti on Sunday at a reception to celebrate the 90th birthday of his mother, Deaconess Felicia Aribisala, also canvassed support from well-meaning Nigerians to the institution, saying, “A technology-based institution of this nature is capital-intensive”.

He said, “The NUC is coming for Resource Verification of all the 36 programmes that we are trying to offer. As soon as they come, by the special grace of God, we have provided those things that will be needed.
“We have provided a modern laboratory for all the programmes. We have a library now. We have classrooms fixed.

“We have offices and furniture fixed. We have all of those things. So we are very confident we are going to scale through.

“By the time we now scale through, by the special grace of God, by September this year, we are going to ask those who are interested in our university to do Change of University, and admission will begin. That is the icing. And after that, recruitment of staff will just follow”.

The VC, who said that funding of education should not be left to the government alone, said, “Universities need a lot of funding. Funding is a major challenge. You have to provide facilities and all of those things.
“So, as I speak to you, we (FUTES) do not have enough funds. That’s why we keep appealing and going to people because the government cannot do it all alone. We have been visiting some people who are public-spirited, people who like education, tertiary education.

“If we have people who want to donate buildings, we are going to name such after them; people who want to give scholarships; people who want to build hostels in such a manner that it is their own and they will take rent and all of those things.

“I think the funding is crucial because if you look at the nature of our university, University of Technology and Environmental Sciences, it is capital-intensive, it is technology-based.

“It means we need a lot of equipment. As I said, the government cannot do everything. So we need help at this time financially,” the VC said.

Aribisala disclosed that the land issue, which could have been a challenge to the university, had been resolved amicably with an agreement made with the concerned families.

“As I speak to you now, it has been resolved. The 200 hectares that have been donated to the university are very intact.

“There has been an agreement. The community and government will also pay some compensation to the families.

“So they are now at peace. The community is not trying to force the land. I think that was the kind of misconception that happened at the time,” the Vice Chancellor said.
"""

In [ ]:
clean = clean_and_merge_article(article)
print(clean)

### **Model Saving and Deployment **

With **epoch 2** identified as the best-performing checkpoint, we save this model for future use.  

The saved model can be:  
- **Reloaded locally** for inference or further fine-tuning  
- **Uploaded to Hugging Face Hub** to make it publicly accessible  
- **Integrated into applications** (e.g., Flask, FastAPI, or Streamlit apps) for real-world summarization tasks  

This ensures that our best model is preserved and can be easily deployed for production-level usage.  


In [ ]:
!pip install huggingface_hub

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint_path = "./bart-news-summarizer/checkpoint-8000"  # <-- your epoch2 path
save_path = "./bart-summarizer-epoch2"  # final folder you’ll save

# load checkpoint
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# save clean copy
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Epoch2 model saved at {save_path}")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)

print("✅ Reloaded epoch2 model successfully")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

repo_id = "Jayywestty/bart-summarizer-epoch2"  # change to your HF username/repo

# load from your already cleaned folder
model = AutoModelForSeq2SeqLM.from_pretrained("./bart-summarizer-epoch2")
tokenizer = AutoTokenizer.from_pretrained("./bart-summarizer-epoch2")

# push to hub
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print(f"✅ Model uploaded to https://huggingface.co/{repo_id}")
